## on fait un BERTopic pour voir les topics et réduire ceux qui sont en trop grand nombre
l'idée c'est de réduire l'imbalance de classe 1 et 0

In [3]:
# 1. Installation des dépendances nécessaires
# pip install bertopic sentence-transformers transformers umap-learn hdbscan scikit-learn pandas numpy tqdm matplotlib

# 2. Importation des bibliothèques
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, precision_recall_curve
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_distances

# 3. Chargement des données
print("Chargement des données...")
train_df = pd.read_csv("data/train_data_SMM4H_2025_Task_1.csv")
test_df = pd.read_csv("data/test_data_SMM4H_2025_Task_1_no_labels.csv")

# Vérification de l'équilibre des classes dans les données d'entraînement
print(f"Distribution des classes dans les données d'entraînement:")
print(train_df['label'].value_counts())
print(f"Pourcentage de positifs: {100 * train_df['label'].mean():.2f}%")

# 4. Division des données d'entraînement en train et validation
train_data, val_data = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=42)

# 5. Création des embeddings avec SentenceTransformer (modèle multilingue)
print("Création des embeddings...")
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

# Fonction pour traiter les données par lots
def create_embeddings_batch(texts, batch_size=32):
    return model.encode(texts, batch_size=batch_size, show_progress_bar=True)

# Création des embeddings pour les ensembles d'entraînement et de validation
train_embeddings = create_embeddings_batch(train_data['text'].tolist())
val_embeddings = create_embeddings_batch(val_data['text'].tolist())

# 6. Configuration et entraînement du modèle BERTopic
print("Configuration du modèle BERTopic...")
# Paramètres UMAP plus adaptés aux petits datasets multilingues
umap_model = UMAP(
    n_neighbors=15,
    n_components=5,
    min_dist=0.0,
    metric='cosine',
    random_state=42
)

# Configuration HDBSCAN pour détecter plus de clusters
hdbscan_model = HDBSCAN(
    min_cluster_size=10,
    min_samples=5,
    prediction_data=True
)

# Initialisation du modèle BERTopic
topic_model = BERTopic(
    embedding_model=None,  # On utilise nos propres embeddings
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    calculate_probabilities=True,
    verbose=True
)

# Entraînement du modèle BERTopic
print("Entraînement du modèle BERTopic...")
topics, probs = topic_model.fit_transform(train_data['text'].tolist(), train_embeddings)

# 7. Ajout des topics aux données d'entraînement
train_data['topic'] = topics

# 8. Stratégie de réduction du déséquilibre des classes
print("Réduction du déséquilibre des classes...")
df_pos = train_data[train_data['label'] == 1]
df_neg = train_data[train_data['label'] == 0]

print(f"Avant réduction: Positifs={len(df_pos)}, Négatifs={len(df_neg)}")

# Calcul du ratio cible pour un meilleur équilibrage
# Viser un ratio de 1:3 (par exemple) au lieu de 1:100
target_ratio = 3  # 3 négatifs pour 1 positif
total_neg_needed = len(df_pos) * target_ratio

# Détermination dynamique du pourcentage à conserver
overall_reduction_ratio = min(1.0, total_neg_needed / len(df_neg))
print(f"Ratio de réduction global: {overall_reduction_ratio:.4f}")

reduced_neg = []
for topic in df_neg['topic'].unique():
    # Sous-ensemble des exemples négatifs pour ce topic
    sub_df = df_neg[df_neg['topic'] == topic]
    
    # Pour les très petits clusters, on garde tout
    if len(sub_df) < 5:
        reduced_neg.append(sub_df)
        continue
    
    # Récupération des embeddings pour ce topic
    indices = sub_df.index.tolist()
    topic_indices = [train_data.index.get_loc(idx) for idx in indices]
    topic_embeddings = train_embeddings[topic_indices]
    
    # Calcul du centroïde du topic
    centroid = np.mean(topic_embeddings, axis=0, keepdims=True)
    
    # Calcul des distances au centroïde
    distances = cosine_distances(topic_embeddings, centroid).flatten()
    
    # Tri des indices par distance
    sorted_indices = np.argsort(distances)
    
    # Ajustement du nombre d'exemples à conserver en fonction de la taille du topic
    # Les topics plus grands contribuent plus à la réduction
    topic_size_ratio = len(sub_df) / len(df_neg)
    topic_reduction_ratio = overall_reduction_ratio / topic_size_ratio
    keep_ratio = min(1.0, topic_reduction_ratio * 1.5)  # Légère préférence pour garder plus d'exemples des petits topics
    
    keep_n = max(1, int(keep_ratio * len(sorted_indices)))
    
    # Sélection des exemples les plus proches du centroïde
    selected = sub_df.iloc[sorted_indices[:keep_n]]
    reduced_neg.append(selected)

# Combinaison de tous les exemples négatifs réduits
df_neg_reduced = pd.concat(reduced_neg)

# Création du dataset final équilibré
df_final = pd.concat([df_pos, df_neg_reduced]).sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Après réduction: Positifs={len(df_pos)}, Négatifs={len(df_neg_reduced)}")
print(f"Nouveau ratio négatif/positif: {len(df_neg_reduced)/len(df_pos):.2f}")

# 9. Sauvegarde du dataset équilibré
df_final.to_csv("data/balanced_dataset.csv", index=False)
print("✅ Dataset équilibré sauvegardé.")

# 10. Visualisation de la distribution des topics
plt.figure(figsize=(12, 6))
topic_counts = train_data['topic'].value_counts()
topic_counts = topic_counts[topic_counts.index != -1]  # Exclure les outliers (-1)
topic_counts.sort_index().plot(kind='bar')
plt.title('Distribution des topics')
plt.xlabel('Topic ID')
plt.ylabel('Nombre de documents')
plt.savefig('topic_distribution.png')
plt.close()

# 11. Exemples de chaque topic pour vérification
print("\nExemples de textes par topic:")
for topic_id in sorted(train_data['topic'].unique())[:5]:  # Afficher les 5 premiers topics
    if topic_id == -1:
        continue  # Sauter les outliers
    topic_texts = train_data[train_data['topic'] == topic_id]['text'].tolist()[:2]
    print(f"\nTopic {topic_id}:")
    for i, text in enumerate(topic_texts):
        print(f"  Exemple {i+1}: {text[:100]}...")  # Afficher les 100 premiers caractères

# 12. Préparation des données pour le test
print("\nPréparation des données de test...")
test_embeddings = create_embeddings_batch(test_df['text'].tolist())

# Assignation des topics aux données de test
test_topics, test_probs = topic_model.transform(test_df['text'].tolist(), test_embeddings)
test_df['topic'] = test_topics

# Sauvegarde des données de test avec leurs topics
test_df.to_csv("data/test_data_with_topics.csv", index=False)
print("✅ Données de test avec topics sauvegardées.")

print("\nTerminé ! Les données équilibrées sont prêtes pour l'entraînement d'un modèle de classification.")

Chargement des données...
Distribution des classes dans les données d'entraînement:
label
0    28736
1     2451
Name: count, dtype: int64
Pourcentage de positifs: 7.86%
Création des embeddings...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

2025-04-09 10:56:32,203 - BERTopic - Dimensionality - Completed ✓
2025-04-09 10:56:32,204 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-09 10:59:24,703 - BERTopic - Cluster - Completed ✓
2025-04-09 10:59:24,710 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-09 10:59:25,231 - BERTopic - Representation - Completed ✓


Réduction du déséquilibre des classes...
Avant réduction: Positifs=1961, Négatifs=22988
Ratio de réduction global: 0.2559
Après réduction: Positifs=1961, Négatifs=21533
Nouveau ratio négatif/positif: 10.98
✅ Dataset équilibré sauvegardé.

Exemples de textes par topic:

Topic 0:
  Exemple 1: i took my trazodone like an hour ago and i can barely keep my eyes open and i need to finish homewor...
  Exemple 2: fuck seroquel. i need to be awake...

Topic 1:
  Exemple 1: Этот препарат рассчитан на детский возраст начиная от двух лет....
  Exemple 2: Нам порекомендовал его на комиссии логопед-дефектолог....

Topic 2:
  Exemple 1: @USER_________ I'm nervous about Remicade because my veins are horrible. But nervous about Humira be...
  Exemple 2: Early night for me, going to be a long day tomo with my first humira injections! Time to finally bea...

Topic 3:
  Exemple 1: Oui, tu viens certainement, pour moi aussi, les cycles ont changé. Au début, ils arrivaient tous les...
  Exemple 2: Salut <us

Batches:   0%|          | 0/726 [00:00<?, ?it/s]

2025-04-09 11:00:00,791 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
c:\Users\Olivier\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\sparse\_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
2025-04-09 11:00:24,620 - BERTopic - Dimensionality - Completed ✓
2025-04-09 11:00:24,621 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2025-04-09 11:00:26,617 - BERTopic - Probabilities - Start calculation of probabilities with HDBSCAN
2025-04-09 11:03:05,887 - BERTopic - Probabilities - Completed ✓
2025-04-09 11:03:05,888 - BERTopic - Cluster - Completed ✓


✅ Données de test avec topics sauvegardées.

Terminé ! Les données équilibrées sont prêtes pour l'entraînement d'un modèle de classification.
